<a href="https://colab.research.google.com/github/Sanskriti-hello/Cynaptics_Induction/blob/main/AivsReal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import os
from PIL import Image
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Define constants
TRAIN_DIR = "/content/drive/MyDrive/Data/Train"
BATCH_SIZE = 25
EPOCHS = 20
IMAGE_SIZE = (236, 236)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 1: Prepare the Dataset
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Load dataset and encode labels
def create_dataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
    return pd.DataFrame({"image": image_paths, "label": labels})

df = create_dataframe(TRAIN_DIR)

# Encode labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create PyTorch datasets
train_dataset = CustomDataset(train_df['image'].values, train_df['label'].values, transform=transform)
val_dataset = CustomDataset(val_df['image'].values, val_df['label'].values, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

# Constants
TRAIN_DIR = "/path/to/train/folder"
TEST_DIR = "/path/to/test/folder"
BATCH_SIZE = 32
EPOCHS = 20
IMAGE_SIZE = (236, 236)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare Training Data
def create_dataframe(train_dir):
    image_paths = []
    labels = []
    for label in os.listdir(train_dir):
        folder_path = os.path.join(train_dir, label)
        if os.path.isdir(folder_path):
            for image_name in os.listdir(folder_path):
                image_paths.append(os.path.join(folder_path, image_name))
                labels.append(label)
    return pd.DataFrame({"image": image_paths, "label": labels})

train_df = create_dataframe(TRAIN_DIR)

# Encode labels
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])  # 'AI' -> 0, 'Real' -> 1

# Define Dataset Class
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None, test=False):
        self.dataframe = dataframe
        self.transform = transform
        self.test = test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx]['image']
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        if self.test:
            return image, os.path.basename(image_path)
        label = self.dataframe.iloc[idx]['label']
        return image, label

# Transformations
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create Training Dataset and DataLoader
train_dataset = ImageDataset(train_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Define the CNN Model
class CNNModel(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * (IMAGE_SIZE[0] // 16) * (IMAGE_SIZE[1] // 16), 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = CNNModel(num_classes=2).to(DEVICE)

# Train the Model
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Metrics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

train_model(model, train_loader, criterion, optimizer, EPOCHS)

# Use Test Data for Predictions

test_images = [os.path.join(TEST_DIR, img) for img in os.listdir(TEST_DIR)]
test_df = pd.DataFrame({"image": test_images})

test_dataset = ImageDataset(test_df, transform=transform, test=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model.eval()
predictions = []


with torch.no_grad():
    for images, image_names in tqdm(test_loader):
        images = images.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted_labels = le.inverse_transform(predicted.cpu().numpy())
        for name, label in zip(image_names, predicted_labels):
            predictions.append({"Id": name, "Label": label})

# Save Predictions
submission_df = pd.DataFrame(predictions)
submission_df.to_csv("submission.csv", index=False)
print("Predictions saved to submission.csv!")


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("splcher/animefacedataset")

print("Path to dataset files:", path)

100%|██████████| 395M/395M [00:08<00:00, 50.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/splcher/animefacedataset/versions/3


In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

TRAIN_DIR = "/content/drive/MyDrive/Data/Train"
TEST_DIR = "/content/drive/MyDrive/Data/Test"
BATCH_SIZE = 25
EPOCHS = 15
IMAGE_SIZE = (236, 236)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def create_dataframe(train_dir):
    image_paths = []
    labels = []
    for label in os.listdir(train_dir):
        folder_path = os.path.join(train_dir, label)
        if os.path.isdir(folder_path):
            for image_name in os.listdir(folder_path):
                image_paths.append(os.path.join(folder_path, image_name))
                labels.append(label)
    return pd.DataFrame({"image": image_paths, "label": labels})

train_df = create_dataframe(TRAIN_DIR)

le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])

class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None, test=False):
        self.dataframe = dataframe
        self.transform = transform
        self.test = test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx]['image']
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        if self.test:
            return image, os.path.basename(image_path)
        label = self.dataframe.iloc[idx]['label']
        return image, label

transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageDataset(train_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

class CNNModel(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * (IMAGE_SIZE[0] // 16) * (IMAGE_SIZE[1] // 16), 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = CNNModel(num_classes=2).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(DEVICE), labels.to(DEVICE)


            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

train_model(model, train_loader, criterion, optimizer, EPOCHS)

test_images = [os.path.join(TEST_DIR, img) for img in os.listdir(TEST_DIR)]

exception_file = "/content/drive/MyDrive/Data/Test/image_62.jpg"
test_images = [img for img in test_images if img != exception_file]

test_df = pd.DataFrame({"image": test_images})

test_dataset = ImageDataset(test_df, transform=transform, test=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model.eval()
predictions = []

with torch.no_grad():
    for images, image_names in tqdm(test_loader):
        images = images.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted_labels = le.inverse_transform(predicted.cpu().numpy())

        for name, label in zip(image_names, predicted_labels):
            predictions.append({"Id": name, "Label": label})


submission_df = pd.DataFrame(predictions)
submission_df.to_csv("submission.csv", index=False)
print("Predictions saved to submission.csv!")








  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 1/15, Loss: 1.4893, Accuracy: 71.06%


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 2/15, Loss: 0.2542, Accuracy: 89.62%


  0%|          | 0/33 [00:00<?, ?it/s]

...

Epoch 14/15, Loss: 0.0685, Accuracy: 98.17%


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 15/15, Loss: 0.0302, Accuracy: 99.63%


  0%|          | 0/8 [00:00<?, ?it/s]

Predictions saved to submission.csv!


In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

# Define constants
TRAIN_DIR = "/content/drive/MyDrive/Data/Train"
TEST_DIR = "/content/drive/MyDrive/Data/Test"
BATCH_SIZE = 25
EPOCHS = 15
IMAGE_SIZE = (236, 236)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to create a dataframe with image paths and labels
def create_dataframe(train_dir):
    image_paths = []
    labels = []
    for label in os.listdir(train_dir):
        folder_path = os.path.join(train_dir, label)
        if os.path.isdir(folder_path):
            for image_name in os.listdir(folder_path):
                image_paths.append(os.path.join(folder_path, image_name))
                labels.append(label)
    return pd.DataFrame({"image": image_paths, "label": labels})

# Create training dataframe and encode labels
train_df = create_dataframe(TRAIN_DIR)
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])

# Dataset class to handle loading and preprocessing
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None, test=False):
        self.dataframe = dataframe
        self.transform = transform
        self.test = test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx]['image']
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        if self.test:
            return image, os.path.basename(image_path)
        label = self.dataframe.iloc[idx]['label']
        return image, label

# Augmentation and preprocessing for training and testing
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create DataLoader for training
train_dataset = ImageDataset(train_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# CNN model definition
class CNNModel(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * (IMAGE_SIZE[0] // 16) * (IMAGE_SIZE[1] // 16), 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Model, loss, and optimizer
model = CNNModel(num_classes=len(le.classes_)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training function
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        scheduler.step()  # Update learning rate
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

# Train the model
train_model(model, train_loader, criterion, optimizer, scheduler, EPOCHS)

# Preparing the test dataset
test_images = [os.path.join(TEST_DIR, img) for img in os.listdir(TEST_DIR)]

exception_file = "/content/drive/MyDrive/Data/Test/image_62.jpg"
test_images = [img for img in test_images if img != exception_file]

test_df = pd.DataFrame({"image": test_images})

test_dataset = ImageDataset(test_df, transform=transform, test=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Inference and saving predictions
model.eval()
predictions = []

with torch.no_grad():
    for images, image_names in tqdm(test_loader):
        images = images.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted_labels = le.inverse_transform(predicted.cpu().numpy())

        for name, label in zip(image_names, predicted_labels):
            predictions.append({"Id": name, "Label": label})

submission_df = pd.DataFrame(predictions)
submission_df.to_csv("submission2.csv", index=False)
print("Predictions saved to submission.csv!")


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 1/15, Loss: 1.9466, Accuracy: 64.22%


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 2/15, Loss: 0.2994, Accuracy: 87.18%


  0%|          | 0/33 [00:00<?, ?it/s]

...

Epoch 14/15, Loss: 0.0399, Accuracy: 98.41%


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 15/15, Loss: 0.0383, Accuracy: 98.41%


  0%|          | 0/8 [00:00<?, ?it/s]

Predictions saved to submission.csv!


In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

# Define constants
TRAIN_DIR = "/content/drive/MyDrive/Data/Train"
TEST_DIR = "/content/drive/MyDrive/Test_Images"
BATCH_SIZE = 25
EPOCHS = 15
IMAGE_SIZE = (236, 236)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to create a dataframe with image paths and labels
def create_dataframe(train_dir):
    image_paths = []
    labels = []
    for label in os.listdir(train_dir):
        folder_path = os.path.join(train_dir, label)
        if os.path.isdir(folder_path):
            for image_name in os.listdir(folder_path):
                image_paths.append(os.path.join(folder_path, image_name))
                labels.append(label)
    return pd.DataFrame({"image": image_paths, "label": labels})

# Create training dataframe and encode labels
train_df = create_dataframe(TRAIN_DIR)
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])

# Dataset class to handle loading and preprocessing
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None, test=False):
        self.dataframe = dataframe
        self.transform = transform
        self.test = test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx]['image']
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        if self.test:
            return image, os.path.basename(image_path)
        label = self.dataframe.iloc[idx]['label']
        return image, label

# Augmentation and preprocessing for training and testing
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create DataLoader for training
train_dataset = ImageDataset(train_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# CNN model definition
class CNNModel(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * (IMAGE_SIZE[0] // 16) * (IMAGE_SIZE[1] // 16), 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Model, loss, and optimizer
model = CNNModel(num_classes=len(le.classes_)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training function
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        scheduler.step()  # Update learning rate
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

# Train the model
train_model(model, train_loader, criterion, optimizer, scheduler, EPOCHS)

# Preparing the test dataset
test_images = [os.path.join(TEST_DIR, img) for img in os.listdir(TEST_DIR)]

exception_file = "/content/drive/MyDrive/Data/Test/image_62.jpg"
test_images = [img for img in test_images if img != exception_file]

test_df = pd.DataFrame({"image": test_images})

test_dataset = ImageDataset(test_df, transform=transform, test=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Inference and saving predictions
model.eval()
predictions = []

with torch.no_grad():
    for images, image_names in tqdm(test_loader):
        images = images.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted_labels = le.inverse_transform(predicted.cpu().numpy())

        for name, label in zip(image_names, predicted_labels):
            predictions.append({"Id": name, "Label": label})

submission_df = pd.DataFrame(predictions)
submission_df.to_csv("submission_final.csv", index=False)
print("Predictions saved to submission.csv!")


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 1/15, Loss: 2.3587, Accuracy: 73.38%


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 2/15, Loss: 0.2929, Accuracy: 87.79%


  0%|          | 0/33 [00:00<?, ?it/s]

...

Epoch 14/15, Loss: 0.0186, Accuracy: 98.90%


  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 15/15, Loss: 0.0123, Accuracy: 99.51%


  0%|          | 0/8 [00:00<?, ?it/s]

Predictions saved to submission.csv!
